In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd

In [2]:
from subprocess import check_output
print(check_output(["ls", "../sign_data"]).decode("utf8"))

full



In [3]:
def show_image(image,text=None, should_save=False):
    np_image = image.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(np_image, (1, 2, 0)))
    plt.show()    

In [4]:
####### THIS IS FOR CREATING THE DATASET, DONT RUN IT BECAUSE I HAVE CREATED IT ALREADY AND YOU WILL MESS IT UP

# Define the base path for signature images
file_path = '../sign_data/full'

# Initialize an empty list to store the data
data_list = []

# List and sort the folders, then select the first 128 folders
folders = sorted(os.listdir(file_path))[:128]

for i in range(0, len(folders), 2):  # Process every second folder
    genuine_folder = folders[i]
    genuine_path = os.path.join(file_path, genuine_folder)
    forgery_path = os.path.join(file_path, genuine_folder + '_forg')
    
    # Generate all pairs of genuine signatures
    genuine_files = os.listdir(genuine_path)
    for y in genuine_files:
        for z in genuine_files:
            if y != z:
                data_list.append([os.path.join(genuine_folder, y), os.path.join(genuine_folder, z), 1])
                
    # Generate pairs of genuine and forged signatures
    if os.path.exists(forgery_path):  # Check if the forgery folder exists
        forged_files = os.listdir(forgery_path)
        for y in genuine_files:
            for z in forged_files:
                data_list.append([os.path.join(genuine_folder, y), os.path.join(genuine_folder + '_forg', z), 0])

# Create a DataFrame from the list
data = pd.DataFrame(data_list, columns=['image1', 'image2', 'label'])

# Save the DataFrame to a CSV file
#data.to_csv('saved_data.csv', index=False)


In [5]:
data = pd.read_csv('saved_data.csv').sample(frac=1) 

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data.iloc[:, :2],  # Selects the first two columns as features
    data.iloc[:, 2],   # Selects the third column as the target variable
    test_size=0.2,     # Allocates 10% of the data for the test set
    stratify=data.iloc[:, 2]  # Ensures the training and testing sets have similar target variable distribution
)

In [6]:
pd.concat([X_train,y_train],axis=1).to_csv('train_data.csv',index=False)
pd.concat([X_test,y_test],axis=1).to_csv('test_data.csv',index=False)